# Processing

In [65]:
import pandas as pd
import numpy as np
import os

In [73]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

swaped_states = {value:key for key, value in us_state_to_abbrev.items()}

## Loading datasets

In [165]:
shootings = pd.read_csv('./datasets/shootings2.csv')
gun_possesion = pd.read_csv('./datasets/gun-ownership.csv')
hdi = pd.read_csv('./datasets/HDI.csv')
shootings['date'] = pd.to_datetime(shootings['date'])

### Concat gun-ownership

In [146]:
def ownership_value(state_tag):
    """ Para cada estado devuelve el valor de posesion de armas de este"""
    try:
        state_name = swaped_states[state_tag]
        return gun_possesion[gun_possesion["State"] == state_name]["gunOwnership"].to_list()[0]
    except:
        return np.nan
    
shooting_v1 = shootings.copy()
shooting_v1["gun-ownership"] = shooting_v1.apply(lambda row : ownership_value(row["state"]), axis=1)
shooting_v1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,gun-ownership
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,WA,True,attack,Not fleeing,False,Guns,0.421
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,OR,False,attack,Not fleeing,False,Guns,0.508
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,KS,False,other,Not fleeing,False,Unarmed,0.489
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,CA,True,attack,Not fleeing,False,Other unusual objects,0.283
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,CO,False,attack,Not fleeing,False,Piercing objects,0.451


In [59]:
print(len(swaped_states))
print(len(new["state"].unique()))

51
51


In [93]:
shooting_v1.to_csv('./datasets/shootings2_v1.csv', index=False)


### Concat HDI

In [174]:
selected_cols = ['Region', '2015', '2016', '2017', '2018', '2019']
hdi_v1 = hdi[selected_cols]

In [175]:
hdi_v1.rename(columns={col: 'hdi_' + col for col in hdi_v1.columns if col in selected_cols[1::]}, inplace=True)

c:\users\arnau\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [176]:
def value_hdi(row):
    anno = row['date'].year
    if anno != 2020:
        state = row["state"]
        state_name = swaped_states[state]
        return hdi_v1[hdi_v1['Region'] == state_name]['hdi_' + str(anno)].to_list()[0]
    return np.nan

shooting_v1["hdi"] = shooting_v1.apply(lambda row : value_hdi(row), axis=1)